In [24]:
#flask version
import flask
print(flask.__version__)

1.1.2


In [25]:
import tensorflow as tf
import pickle
model = pickle.load(open('model.pkl','rb'))
from flask import Flask, request, jsonify, render_template

In [26]:
#read_image
IMG_DIMEN = 224
N_COLORS = 3

In [27]:
import cv2
import numpy as np

def read_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=(IMG_DIMEN, IMG_DIMEN))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype('float32')/255.0
    return img

In [28]:
#visualizing an image
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):
    plt.figure(figsize=(6, 6))
    if N_COLORS == 3:
        plt.imshow(img)
    elif N_COLORS == 1:
         plt.imshow(img, cmap='gray')
    plt.show()

In [29]:
#show_image
#img =read_image('D:/xampp/htdocs/xampp/DEPI-Project/chest-ray/chest_xray/test/PNEUMONIA/person1679_virus_2896.jpeg')
#imshow(img)

In [36]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten,Dropout,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
base_model=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

for layer in base_model.layers:
  layer.trainable=False

x=base_model.output
x=Flatten()(x)
x=Dense(256,activation='relu')(x)
x=Dropout(0.5)(x)
output_vgg=Dense(1,activation='sigmoid')(x)
model_vgg=Model(inputs=base_model.input,outputs=output_vgg)
# compile model
model_vgg.compile(optimizer=Adam(learning_rate=0.00001),loss='binary_crossentropy',metrics=['accuracy'])

In [31]:
#load saved_model
from tensorflow.keras.models import load_model

In [32]:
#model_vgg.load_weights('D:/Xampp/htdocs/xampp/DEPI-Project/chest-ray/Saved-model/pneumonia_detection.h5')

In [33]:
#classify_image
def classify_image(img_path):
    img = read_image(img_path)
    return float(model_vgg.predict(np.array([img])).squeeze())

In [34]:
#classify_image('D:/xampp/htdocs/xampp/DEPI-Project/chest-ray/chest_xray/test/PNEUMONIA/person1679_virus_2896.jpeg')

In [40]:
#flask_API
from flask import Flask
from flask import request

app = Flask(__name__)

@app.route('/classify', methods = ['POST'])
def classify_handler():
    img_path = request.form['path']
    y_pred = classify_image(img_path)
    class_name = 'PNEUMONIA' if y_pred >= 0.5 else 'NORMAL'
    prob_PNEUMONIA = y_pred
    prob_NORMAL = 1 - y_pred
    print('img_path:', img_path)
    print('y_pred:', y_pred)
    print('class_name:', class_name)
    return {
        'prob_NORMAL': prob_NORMAL, 
        'prob_PNEUMONIA': prob_PNEUMONIA, 
        'class_name': class_name
    }
@app.route('/predict', methods=['POST'])
def predict():
    float_features = [float(x) for x in request.form.values()]
    final_features = [np.array(float_features)]
    prediction = model.predict(final_features)
   
    output = round(prediction[0])
    if output == 0:
        message = 'The person does not have cancer'
        print(message)
    else:
        message = 'The person has cancer'
        print(message)
    #return render_template('index.html',prediction_text="The Perdict is {}".format(output))
    return render_template('cancer_Result.html',prediction_text=message)

app.run(port=8000,debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
